코랩에서 돌렸습니다.

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

In [8]:
import pandas as pd
import os
from tqdm import tqdm


# 팀명과 소환사명이 공백 없이 붙어서 나와 문제가 생긴 데이터들이 일부 있습니다. 그걸 팀명과 소환사명으로 분리하기 위한 코드입니다.
# 각 팀 5명의 선수에 대해 공통적으로 앞에 붙은 문자열은 팀명으로 간주합니다.
# 일치하는 부분이 없으면 "", 0을 리턴합니다. 파라미터가 문자열이 아니면 "", -1을 리턴합니다.

def find_common_prefix(strings):
    if not strings:
        return "", -1

    # 짧은 문자 찾기
    shortest = min(strings, key=len)

    # 짧은 문자의 가장 긴 것으로 max값을 잡아 놓고
    end_index = len(shortest)

    for i in range(len(shortest)):  # 문자열의 첫 글자부터
        current_char = shortest[i]
        for string in strings:
            if string[i] != current_char:
                end_index = i
                break
        if end_index != len(shortest):
            break

    common_prefix = shortest[:end_index]
    return common_prefix, end_index

In [9]:
#COLLECTED_DATA_PATH = "/content/drive/MyDrive/penta_kill/data science/collected_game_data/"
COLLECTED_DATA_PATH = "../../data/collected_data/"
#COLLECTED_DATA_PATH = "../../data/collected_data_test/"
file_list = os.listdir(COLLECTED_DATA_PATH)

In [10]:
#file_list[221]  # 이름이 500이라 버그난곳

In [11]:
PARTICIPANTS_NUMBER_OF_A_TEAM = 5

count = 0

for file in tqdm(file_list):
  blueSummonerNames = []
  redSummonerNames = []
  needed = True
  if file.find("_invalid") == -1 and file.find(".xlsx") != -1:
    df = pd.read_excel(COLLECTED_DATA_PATH + file)
    df = df.astype({"esportsTeamId_Blue":"str", "esportsTeamId_Red":"str", "esportsPlayerId_0":"str", "esportsPlayerId_1":"str", "esportsPlayerId_2":"str", "esportsPlayerId_3":"str", "esportsPlayerId_4":"str", "esportsPlayerId_5":"str", "esportsPlayerId_6":"str", "esportsPlayerId_7":"str", "esportsPlayerId_8":"str", "esportsPlayerId_9":"str"})

    for i in range(PARTICIPANTS_NUMBER_OF_A_TEAM):
      blueteam = str(df.at[0, f"teamCode_{i}"])
      bluename = str(df.at[0, f"summonerName_{i}"])
      redteam = str(df.at[0, f"teamCode_{i + PARTICIPANTS_NUMBER_OF_A_TEAM}"])
      redname = str(df.at[0, f"summonerName_{i+PARTICIPANTS_NUMBER_OF_A_TEAM}"])
      if len(blueteam) == 0 or len(bluename) == 0 or len(redteam) == 0 or len(redname) == 0:    # 플레이어 이름이 숫자라 int64타입으로 오는 녀석도 있음 (i.e. 500 걍 이름이 500임)
        needed = False
        continue
      if blueteam != bluename[:-1]: needed = False
      blueSummonerNames.append(bluename)

      if redteam != redname[:-1]: needed = False
      redSummonerNames.append(redname)

    if needed == True:
      count += 1
      print(file, "is needed")
      blueteamName, blueteamWhere = find_common_prefix(blueSummonerNames)
      redteamName, redteamWhere = find_common_prefix(redSummonerNames)

      for i in range(PARTICIPANTS_NUMBER_OF_A_TEAM):
        if blueteamWhere > 0:
          df[f"teamCode_{i}"] = blueteamName
          df[f"summonerName_{i}"] = df[f"summonerName_{i}"].apply(lambda x : x[redteamWhere:])
        if redteamWhere > 0:
          df[f"teamCode_{i+PARTICIPANTS_NUMBER_OF_A_TEAM}"] = redteamName
          df[f"summonerName_{i+PARTICIPANTS_NUMBER_OF_A_TEAM}"] = df[f"summonerName_{i+PARTICIPANTS_NUMBER_OF_A_TEAM}"].apply(lambda x : x[redteamWhere:])
      df.to_excel(COLLECTED_DATA_PATH + file)
print(count, "files is fixed!")

  0%|          | 0/1071 [00:00<?, ?it/s]

100%|██████████| 1071/1071 [17:37<00:00,  1.01it/s]

0 files is fixed!
